## This Notebook will Processing Tags Data in Streaming Using Structured Streaming
Spark Version 3.1.2
Databricks Runtime LTS 9.1

In [0]:
from pyspark.sql.functions import to_date, col, date_format, current_date, from_unixtime, unix_timestamp
from pyspark.sql.types import TimestampType, DateType
from pyspark import Row
from delta.tables import *
import logging
import datetime 

### Common utility functions

In [0]:
%run ../util/utils

In [0]:
tags_filepath="dbfs:/FileStore/tables/asos_data/tags/"
checkpoint_path="dbfs:/FileStore/tables/asos_delta_std/tags_checkpoint"
tags_outpath="dbfs:/FileStore/tables/asos_delta_std/tags/"

### Input Raw folder structure

In [0]:
display(dbutils.fs.ls(tags_filepath))

path,name,size
dbfs:/FileStore/tables/asos_data/tags/20220306/,20220306/,0


In [0]:
####Getting Rating Schema from latest file path
tags_schema_filepath = get_latest_file_path(tags_filepath)
tags_schema = get_schema(tags_schema_filepath, "csv")

In [0]:
df_tags = spark.readStream.format("csv").schema(tags_schema).option("header", True).option("inferSchema", "true").load(tags_filepath + "/*")

In [0]:
df_tags.writeStream.format("delta").outputMode("append").trigger(once=True).option('checkpointLocation', checkpoint_path).start(tags_outpath)

Out[9]: <pyspark.sql.streaming.StreamingQuery at 0x7f110dec8bb0>

In [0]:
display(spark.read.format("delta").load(tags_outpath).limit(10))

genre_id,genre_name
0,unknown
1,Action
2,Adventure
3,Animation
4,Children's
5,Comedy
6,Crime
7,Documentary
8,Drama
9,Fantasy


### Below are the output folders with the checkpoint in place

In [0]:
display(dbutils.fs.ls("/FileStore/tables/asos_delta_std"))

path,name,size
dbfs:/FileStore/tables/asos_delta_std/movies/,movies/,0
dbfs:/FileStore/tables/asos_delta_std/movies_checkpoint/,movies_checkpoint/,0
dbfs:/FileStore/tables/asos_delta_std/rating/,rating/,0
dbfs:/FileStore/tables/asos_delta_std/rating_checkpoint/,rating_checkpoint/,0
dbfs:/FileStore/tables/asos_delta_std/tags/,tags/,0
dbfs:/FileStore/tables/asos_delta_std/tags_checkpoint/,tags_checkpoint/,0
